In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy

## initialize brat reader

In [2]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [3]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [4]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read eHOST annotations | load from pickles

In [5]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [6]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


In [7]:
len(train_docs), len(test_docs)

(303, 202)

## CRF Wrapper (only use for eval)

In [8]:
from CRFWrapper_Sentence import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel


In [9]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Strength', 'ADE', 'Reason', 'Drug', 'Frequency', 'Route', 'Dosage', 'Form', 'Duration'}


In [10]:
crf_model=CRFModel(anno_types=annos)

## converting docs into sentence level dataframe

In [11]:
from ALLSampler_Sentence import SamplingSimulator, ModelSamplingSimulator, VBSamplingSimulator, convert_docs_medspacyIOvec

In [12]:
# get sentence label from docs
sdf_labels_train=convert_docs_medspacyIOvec(train_docs)
sdf_labels_train

,sentence,concept,y,doc_name
0,[**2078-8-9**] Sex: M\n\nService...,Vicodin,Drug,100035.txt
1,"While at the OSH, he received CTX,\nazithromyc...",CTX,Drug,100035.txt
2,"While at the OSH, he received CTX,\nazithromyc...",azithromycin,Drug,100035.txt
3,"While at the OSH, he received CTX,\nazithromyc...",epinephrine,Drug,100035.txt
4,"While at the OSH, he received CTX,\nazithromyc...",solumedrol,Drug,100035.txt
...,...,...,...,...
90405,[**Name (NI) **],,NEG,198406.txt
90406,"[**Telephone/Fax (1) 92787**](H),",,NEG,198406.txt
90407,[**Telephone/Fax (1) 92788**](C)\n\n\nMedicati...,,NEG,198406.txt
90408,3.,,NEG,198406.txt


In [13]:
# get token level label from docs
_, train_df=convert_docs(train_docs, anno_types=annos)
_, test_df=convert_docs(test_docs, anno_types=annos)

In [14]:
train_df

,sentence_id,doc_name,token,label,sentence
0,0,100035.txt,Admission,O,Admission Date:
1,0,100035.txt,Date,O,Admission Date:
2,0,100035.txt,:,O,Admission Date:
3,1,100035.txt,[,O,[**2115-2-22**] Discharge Date: ...
4,1,100035.txt,*,O,[**2115-2-22**] Discharge Date: ...
...,...,...,...,...,...
3151,929927,198406.txt,Followup,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...
3152,929927,198406.txt,Instructions,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...
3153,929927,198406.txt,:,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...
3154,929927,198406.txt,\n,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...


In [15]:
# embedding for unique sentence
pickle_embedding_file= r'..\data\n2c2\embedding_df_uniqueSentID.joblib' 
embedding_df=joblib.load(pickle_embedding_file)

In [16]:
embedding_df

,sentence_id,sentence,embedding
0,0,Admission Date:,"[0.026282, 0.03218903, -0.022386529, 0.0493732..."
3,1,[**2115-2-22**] Discharge Date: ...,"[0.016159855, 0.042264156, -0.018290585, -0.05..."
35,2,[**2078-8-9**] Sex: M\n\nService...,"[0.025958579, -0.05749655, 0.012378361, -0.009..."
113,3,[**Known lastname 3234**] is a 36 year old gen...,"[0.023170307, 0.03989108, 0.026217388, -0.0272..."
163,4,The patient initially presented to LGH ED with...,"[0.008176211, -0.06342948, 0.048615105, -0.045..."
...,...,...,...
3059,929923,"Cyanocobalamin 1,000 mcg/mL Injection once a m...","[0.050521564, -0.08905716, -0.0019493615, -0.0..."
3071,929924,"Lorazepam 0.25 QAM, O.25 QPM, 0.5 mg QHS\n8 Ca...","[-0.030010266, -0.062390286, 0.00167252, 0.016..."
3086,929925,Cream Topical TID\n9.,"[0.026732022, -0.04987913, 0.024520764, -0.016..."
3092,929926,Acetaminophen 1000 mg PO Q6H\n10.,"[-0.017295217, -0.10513715, -0.0030776137, -0...."


In [17]:
sdf_labels_sid = sdf_labels_train.merge(embedding_df, how='inner', on='sentence') 

In [18]:
sdf_labels_sid

,sentence,concept,y,doc_name,sentence_id,embedding
0,[**2078-8-9**] Sex: M\n\nService...,Vicodin,Drug,100035.txt,2,"[0.025958579, -0.05749655, 0.012378361, -0.009..."
1,"While at the OSH, he received CTX,\nazithromyc...",CTX,Drug,100035.txt,5,"[0.038356796, -0.054362558, 0.028156247, -0.02..."
2,"While at the OSH, he received CTX,\nazithromyc...",azithromycin,Drug,100035.txt,5,"[0.038356796, -0.054362558, 0.028156247, -0.02..."
3,"While at the OSH, he received CTX,\nazithromyc...",epinephrine,Drug,100035.txt,5,"[0.038356796, -0.054362558, 0.028156247, -0.02..."
4,"While at the OSH, he received CTX,\nazithromyc...",solumedrol,Drug,100035.txt,5,"[0.038356796, -0.054362558, 0.028156247, -0.02..."
...,...,...,...,...,...,...
638687,Patient had some cardiac enzyme leaks\nduring ...,,NEG,198406.txt,929907,"[0.02065785, -0.06587324, 0.055154495, 0.01074..."
638688,Patient was given cardiac healthy diet during ...,,NEG,198406.txt,929910,"[0.030450102, -0.042418838, 0.00325665, 0.0384..."
638689,# CODE: DNR/DNI (discussed with patient and so...,,NEG,198406.txt,929911,"[0.023342747, 0.013347558, -0.01095362, -0.052..."
638690,"[**Telephone/Fax (1) 92787**](H),",,NEG,198406.txt,929914,"[0.02003492, 0.029056935, -0.0140215475, 0.007..."


In [19]:
train_sentID_list = train_df['sentence_id'].to_list()
train_sentID_set = set(train_sentID_list)
train_sentID_uniqList = list(train_sentID_set)
test_sentID_list = test_df['sentence_id'].to_list()
test_sentID_set = set(test_sentID_list)
test_sentID_uniqList = list(test_sentID_set)
print(len(train_sentID_uniqList), len(test_sentID_uniqList))

51798 34334


In [20]:
# 10 round, how many sample in each round
int(1.0*len(train_sentID_uniqList)/10)

5179

In [21]:
# only input some columns from sentence label dataframe
sdf_labels_sid[['sentence','concept', 'y', 'doc_name','sentence_id']]

,sentence,concept,y,doc_name,sentence_id
0,[**2078-8-9**] Sex: M\n\nService...,Vicodin,Drug,100035.txt,2
1,"While at the OSH, he received CTX,\nazithromyc...",CTX,Drug,100035.txt,5
2,"While at the OSH, he received CTX,\nazithromyc...",azithromycin,Drug,100035.txt,5
3,"While at the OSH, he received CTX,\nazithromyc...",epinephrine,Drug,100035.txt,5
4,"While at the OSH, he received CTX,\nazithromyc...",solumedrol,Drug,100035.txt,5
...,...,...,...,...,...
638687,Patient had some cardiac enzyme leaks\nduring ...,,NEG,198406.txt,929907
638688,Patient was given cardiac healthy diet during ...,,NEG,198406.txt,929910
638689,# CODE: DNR/DNI (discussed with patient and so...,,NEG,198406.txt,929911
638690,"[**Telephone/Fax (1) 92787**](H),",,NEG,198406.txt,929914


## Define sampling simulator

In [22]:
logger.remove()
logger.add(sys.stderr, level='INFO')
logger.add(sys.stderr, level='DEBUG')

2

In [23]:
faiss_index_path= r'..\data\n2c2\faiss_index_st768' #FAISS indexing regenerated for unique sentence embedding

In [27]:
pickle_embedding_file=  r'..\data\n2c2\embedding_df_uniqueSentID.joblib' #unique sentence embeddings
embedding_df=joblib.load(pickle_embedding_file)
vb_simulator=VBSamplingSimulator(total_sents=train_df, 
                                 total_round=10, 
                                 modelWrapper=crf_model, 
                                 eval_sents=test_df, 
                                 init_seed=14, 
                                 faiss_index_path=faiss_index_path, 
                                 embedding_df=embedding_df,
                                 sdf_labels= sdf_labels_sid[['sentence','concept', 'y', 'doc_name','sentence_id']],
                                 min_dist_diff=True
                                )

2024-05-16 23:25:55.808 | DEBUG    | ALLSampler_Sentence:__init__:69 - num per found unique sent: 5179
2024-05-16 23:25:55.815 | DEBUG    | ALLSampler_Sentence:__init__:224 - Loading index...
2024-05-16 23:25:56.817 | DEBUG    | ALLSampler_Sentence:__init__:226 - done


## run test

In [ ]:
scores=vb_simulator.simulate_rounds(boostrap_times=3)

In [ ]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'pl':[], 'pu': [], 'recall': [], 'rl':[], 'ru': [], 'f1':[], 'fl':[], 'fu': []}
for s in scores:    
    for k,v in s.items():
        ave, (l, u)=compute_mean_ci(v)
        summary[k].append(ave)
        summary[k[0]+'l'].append(l)
        summary[k[0]+'u'].append(u)

In [ ]:
pd.options.display.float_format='{:,.5f}'.format
pd.DataFrame(summary)

## bootstrap 3 runs

In [25]:
logger.remove()
logger.add(sys.stderr, level='INFO')
logger.add(sys.stderr, level='DEBUG')

4

In [28]:
boostrap_runs=3
total_round=10

In [29]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [30]:
all_scores=[]
embedding_df=joblib.load(pickle_embedding_file)
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    pickle_embedding_file= r'..\data\n2c2\embedding_df_uniqueSentID.joblib' 
    crf_model=CRFModel(anno_types=annos)
    vb_simulator=VBSamplingSimulator(total_sents=train_df, 
                                 total_round=10, 
                                 modelWrapper=crf_model, 
                                 eval_sents=test_df, 
                                 init_seed=seed, 
                                 faiss_index_path=faiss_index_path, 
                                 embedding_df=embedding_df,
                                 sdf_labels=sdf_labels_sid[['sentence','concept', 'y', 'doc_name','sentence_id']],
                                 min_dist_diff=True
                                )
    #v_simulator=VBSamplingSimulator(train_docs, 
    #                                total_round=10, 
    #                                modelWrapper=crf_model, 
    #                                eval_docs=test_docs, 
    #                                init_seed=seed, 
    #                                faiss_index_path=faiss_index_path, 
    #                                embedding_df=embedding_df, 
    #                                min_dist_diff=True)
    scores=vb_simulator.simulate_rounds(boostrap_times=200)
    all_scores.append(scores) 

2024-05-16 23:27:58.689 | INFO     | __main__:<module>:4 - start run 0.
2024-05-16 23:27:58.689 | INFO     | __main__:<module>:4 - start run 0.
2024-05-16 23:27:58.846 | DEBUG    | ALLSampler_Sentence:__init__:69 - num per found unique sent: 5179
2024-05-16 23:27:58.854 | DEBUG    | ALLSampler_Sentence:__init__:224 - Loading index...
2024-05-16 23:27:59.333 | DEBUG    | ALLSampler_Sentence:__init__:226 - done
2024-05-16 23:27:59.469 | INFO     | ALLSampler_Sentence:simulate_rounds:95 - simulate round 0.
2024-05-16 23:27:59.469 | INFO     | ALLSampler_Sentence:simulate_rounds:95 - simulate round 0.
2024-05-16 23:27:59.472 | DEBUG    | ALLSampler_Sentence:keep_sample:78 - The first round sampling will be random
2024-05-16 23:27:59.883 | INFO     | ALLSampler_Sentence:keep_sample:85 - current sampled sentences: 5179, remaining sentences: 46619
2024-05-16 23:27:59.883 | INFO     | ALLSampler_Sentence:keep_sample:85 - current sampled sentences: 5179, remaining sentences: 46619
2024-05-16 23

In [31]:
joblib.dump(all_scores, r'../data/n2c2/scores_sentence_sampling/ner_VBmin_scores_sentenceSampling_fixed.joblib')


['../data/n2c2/scores_sentence_sampling/ner_VBmin_scores_sentenceSampling_fixed.joblib']